In [1]:
from src import init_data
from arqmath_code.topic_file_reader import Topic
from arqmath_code.Entities.Post import Post
import re
import pyterrier as pt
from bs4 import BeautifulSoup
import copy

In [2]:
topic_reader, data_reader = init_data(task=1)

reading users
reading comments
reading votes
reading post links
reading posts


In [ ]:
data_reader_copy = copy.deepcopy(data_reader)

In [3]:
topic_reader_copy = copy.deepcopy(topic_reader)

In [4]:
test_topic: Topic = topic_reader_copy.get_topic('A.301')
print(test_topic.question)

<p>Suppose <span class="math-container" id="q_2">$A$</span> is a <span class="math-container" id="q_3">$m\times n$</span> matrix.</p> <p>Then Prove that, <span class="math-container" id="q_7">$\begin{equation*} \|A\|_2\leq \sqrt{\|A\|_1 \|A\|_{\infty}} \end{equation*}$</span></p> <p>I have proved the following relations: <span class="math-container" id="q_8">$\begin{align*} \frac{1}{\sqrt{n}}\|A\|_{\infty}\leq \|A\|_2\leq\sqrt{m}\|A\|_{\infty}\\ \frac{1}{\sqrt{m}}\|A\|_{1}\leq \|A\|_2\leq\sqrt{n}\|A\|_{1} \end{align*}$</span> Also I feel that somehow Holder's inequality for the special case when <span class="math-container" id="q_4">$p=1$</span> and <span class="math-container" id="q_5">$q=\infty$</span> might be useful.But I couldn't prove that.</p> <p>Edit: I would like to have a prove that do not use the information that <span class="math-container" id="q_6">$\|A\|_2=\sqrt{\rho(A^TA)}$</span></p> <p>Usage of inequalities like Cauchy Schwartz or Holder is fine.</p> 


In [7]:
topic_copy: Topic = Topic(None, "", "", [])
topic_copy.title = BeautifulSoup(test_topic.title).text
topic_copy.question = BeautifulSoup(test_topic.question).text
topic_copy.question

"Suppose $A$ is a $m\\times n$ matrix. Then Prove that, $\\begin{equation*} \\|A\\|_2\\leq \\sqrt{\\|A\\|_1 \\|A\\|_{\\infty}} \\end{equation*}$ I have proved the following relations: $\\begin{align*} \\frac{1}{\\sqrt{n}}\\|A\\|_{\\infty}\\leq \\|A\\|_2\\leq\\sqrt{m}\\|A\\|_{\\infty}\\\\ \\frac{1}{\\sqrt{m}}\\|A\\|_{1}\\leq \\|A\\|_2\\leq\\sqrt{n}\\|A\\|_{1} \\end{align*}$ Also I feel that somehow Holder's inequality for the special case when $p=1$ and $q=\\infty$ might be useful.But I couldn't prove that. Edit: I would like to have a prove that do not use the information that $\\|A\\|_2=\\sqrt{\\rho(A^TA)}$ Usage of inequalities like Cauchy Schwartz or Holder is fine. "

In [12]:
potential_questions = [question for tag in test_topic.lst_tags for question in data_reader.get_question_of_tag(tag=tag)]
len(potential_questions)
potential_questions[0].body

'<p>I was reading up on the Fibonacci Sequence,  <span class="math-container" id="336">\\text {{1,1,2,3,5,8,13,....}}</span> when I\'ve noticed some were able to calculate specific numbers. So far I\'ve only figured out creating an array and counting to the value, which is incredibly simple, but I reckon I can\'t find any formula for calculating a Fibonacci number based on it\'s position.</p>  <p>Is there a way to do this? If so, how are we able to apply these formulas to arrays?</p> '

In [ ]:
for question in potential_questions:
    question.title = BeautifulSoup(question.title).text
    question.body = BeautifulSoup(question.body).text

potential_questions[0].body

In [27]:
potential_questions[0].__class__

arqmath_code.Entities.Post.Question

In [14]:
if not pt.started():
  pt.init()

terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /Users/I518152/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /Users/I518152/.pyterrier...
Done
         Using cpu = 'i386' instead!


PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [15]:
import pandas as pd

# lets not truncate output too much
pd.set_option('display.max_colwidth', 150)

docs_df = pd.DataFrame([
        ["d1", "this is the first document of many documents"],
        ["d2", "this is another document"],
        ["d3", "the topic of this document is unknown"]
    ], columns=["docno", "text"])

docs_df

,docno,text
0,d1,this is the first document of many documents
1,d2,this is another document
2,d3,the topic of this document is unknown


In [ ]:
indexer = pt.DFIndexer("./index_3docs", overwrite=True)
index_ref = indexer.index(docs_df["text"], docs_df["docno"])
index_ref.toString()

In [ ]:
indexer_arqmath = pt.IterDictIndexer("./index_arqmathV1")
index_ref_arqmath = indexer_arqmath.index(potential_questions) #not working but right approach, further preprocessing needed to create a list of dicts
index_ref_arqmath.toString()

In [17]:
index = pt.IndexFactory.of(index_ref)

#lets see what type index is.
type(index)

jnius.reflect.org.terrier.structures.Index

In [18]:
print(index.getCollectionStatistics().toString())

Number of documents: 3
Number of terms: 4
Number of postings: 6
Number of fields: 0
Number of tokens: 7
Field names: []
Positions:   false



In [19]:
for kv in index.getLexicon():
  print("%s (%s) -> %s (%s)" % (kv.getKey(), type(kv.getKey()), kv.getValue().toString(), type(kv.getValue()) ) )

document (<class 'str'>) -> term0 Nt=3 TF=4 maxTF=2 @{0 0 0} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
first (<class 'str'>) -> term1 Nt=1 TF=1 maxTF=1 @{0 0 7} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
topic (<class 'str'>) -> term2 Nt=1 TF=1 maxTF=1 @{0 1 1} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
unknown (<class 'str'>) -> term3 Nt=1 TF=1 maxTF=1 @{0 1 5} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)


In [20]:
index.getLexicon()["document"].toString()

'term0 Nt=3 TF=4 maxTF=2 @{0 0 0}'

In [25]:
br = pt.BatchRetrieve(index, wmodel="TF_IDF")
br.search("first")

,qid,docid,docno,rank,score,query
0,1,0,d1,0,0.976744,first
